<a href="https://colab.research.google.com/github/simasaadi/canada-climate-80yrs/blob/main/04_extremes_indices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>